In [1]:
import numpy as np
import pandas as pd 
from  sklearn.preprocessing  import LabelEncoder 
import warnings 
import matplotlib.pyplot as plt
%matplotlib inline 
warnings.filterwarnings('ignore')

In [2]:
train  =pd.read_csv( 'train_lightgbm.csv' )
test   =pd.read_csv( 'test_lightgbm.csv')

In [3]:
test.drop('label' ,axis =1 ,inplace =True )

In [4]:
for feature in train.columns:
    lbe = LabelEncoder()
    train[feature] = lbe.fit_transform(train[feature])
    

In [5]:
for feature in test.columns:
    lbe = LabelEncoder()
    test[feature] = lbe.fit_transform(test[feature])

In [6]:
from  sklearn.model_selection import train_test_split 
X= train.drop('label' ,axis =1 )
y= train['label']
x_train ,x_valid, y_train, y_valid = train_test_split(X ,y ,test_size = 0.2 ,
                                                      random_state =17 )

In [7]:
from  sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

folds = KFold(n_splits=5, shuffle=True, random_state=17)

In [8]:
oof_xgb= np.zeros(len(X))
predictions_xgb= np.zeros(len(test))

In [10]:
##### xgb
import xgboost as xgb

xgb_params = {"booster":'gbtree',
              'eta': 0.005,
              'max_depth': 15,
              'subsample': 0.7, 
              'colsample_bytree': 0.8, 
              'objective': 'binary:logistic', 
              'eval_metric': 'logloss', 
              'silent': True,
              'nthread': 4}##### xgb

folds = KFold(n_splits=5, shuffle=True, random_state=17)
oof_xgb = np.zeros(len(X))
predictions_xgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print("fold n°{}".format(fold_+1))
    trn_data = xgb.DMatrix(X.iloc[trn_idx ,:], y.iloc[trn_idx])
    val_data = xgb.DMatrix(X.iloc[val_idx,: ], y.iloc[val_idx])
    
    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, 
                    num_boost_round=1000,
                    evals=watchlist, 
                    early_stopping_rounds=500, 
                    verbose_eval=100,
                    params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(X.iloc[val_idx ,:]), 
                        ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test),
                    ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, y)))


fold n°1
[0]	train-logloss:0.68993	valid_data-logloss:0.69022
Multiple eval metrics have been passed: 'valid_data-logloss' will be used for early stopping.

Will train until valid_data-logloss hasn't improved in 500 rounds.
[100]	train-logloss:0.50152	valid_data-logloss:0.50846
[200]	train-logloss:0.40949	valid_data-logloss:0.42360
[300]	train-logloss:0.36129	valid_data-logloss:0.38304
[400]	train-logloss:0.33251	valid_data-logloss:0.36218
[500]	train-logloss:0.31435	valid_data-logloss:0.35180
[600]	train-logloss:0.30155	valid_data-logloss:0.34660
[700]	train-logloss:0.29197	valid_data-logloss:0.34423
[800]	train-logloss:0.28438	valid_data-logloss:0.34325
[900]	train-logloss:0.27800	valid_data-logloss:0.34286
[999]	train-logloss:0.27272	valid_data-logloss:0.34277
fold n°2
[0]	train-logloss:0.68990	valid_data-logloss:0.69022
Multiple eval metrics have been passed: 'valid_data-logloss' will be used for early stopping.

Will train until valid_data-logloss hasn't improved in 500 rounds.
[1

In [17]:
# predictions_xgb
y_hat =[]
for  y in oof_xgb:
    if y>0.5:
        y_hat.append(1) 
    else :
        y_hat.append(0)

In [18]:
from sklearn.metrics import accuracy_score 
y = train['label'].values
accuracy_score( y, y_hat ) # 训练集合

0.859354

In [32]:
test['label'] = predictions_xgb
test[['sid'  , 'label']].to_csv('submit_xgb_probility.csv' ,index =False )
test['label']  = test['label'] .map( lambda x : 1 if x>0.5 else 0 )
y_hat = test [['sid' ,'label']].sort_values(by ='sid')['label'].values

test_ans = pd.read_csv("../test1_ans.csv")
y_true = test_ans.sort_values(by ='sid')['label'].values
test[['sid'  , 'label']].to_csv('submit_xgb.csv' ,index =False )

In [20]:
accuracy_score( y_true , y_hat )

0.84772

In [21]:
oof_xgb

array([0.6572606 , 0.08379523, 0.61199534, ..., 0.09402318, 0.04795125,
       0.15331854])

In [25]:
SID_TRAIN = X['sid']
SID_TRAIN

0         219334
1          92432
2         277667
3          46490
4         199784
           ...  
499995    272907
499996    240204
499997    349278
499998    165107
499999    186653
Name: sid, Length: 500000, dtype: int64

In [33]:
train_result = pd.DataFrame()
train_result['sid']= SID_TRAIN
train_result['label'] = oof_xgb
train_result.to_csv("train_xgb_result.csv" ,index =False )